In [9]:
import json
import language_tool_python
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np

In [2]:
tool = language_tool_python.LanguageTool('en-US')

In [3]:
text = 'A sentence with a error in the Hitchhiker’s Guide tot he Galaxy'
matches = tool.check(text)
len(matches)

2

In [4]:
fname = '/shared/0/projects/research-jam-summer-2024/data/english_only/100k_results/wildchat_subset_en_100k_Llama-3.1-70B.jsonl'
cid_to_text = {}
cid_order = []
with open(fname) as f:
    for line in tqdm(f, total=100_000):
        j = json.loads(line)
        cid = j['conversation_hash']
        cid_order.append(cid)
        text = j['human_turn_1']
        cid_to_text[cid] = text

  0%|          | 0/100000 [00:00<?, ?it/s]

In [5]:
cid_to_num_errors = {}

In [12]:
#num_errors = []
for cid, text in tqdm(cid_to_text.items(), total=len(cid_to_text)):
    if cid in cid_to_num_errors:
        continue
    try:
        matches = tool.check(text)
        ne = len(matches)
    except BaseException as e:
        ne = float('nan')
        print(repr(e))
    cid_to_num_errors[cid] = ne    

    # num_errors.append(len(matches))

  0%|          | 0/99440 [00:00<?, ?it/s]

LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0.0.1', port=8082): Read timed out. (read timeout=300)")
LanguageToolError("http://127.0.0.1:8082/v2/: HTTPConnectionPool(host='127.0

In [13]:
len(cid_to_num_errors)

99440

In [14]:
tmp = defaultdict(list)
for cid, ne in cid_to_num_errors.items():
    tmp['conversation_hash'].append(cid)
    tmp['num_grammar_tool_suggestions'].append(ne)
tmp = pd.DataFrame(tmp)

In [16]:
outdir = '/shared/0/projects/research-jam-summer-2024/data/english_only/100k_results/with_metrics/'

In [17]:
tmp.to_csv(outdir + 'human-turn1.grammar-tool-output.csv', index=False)

In [18]:
outdir + 'human-turn1.grammar-tool-output.csv'

'/shared/0/projects/research-jam-summer-2024/data/english_only/100k_results/with_metrics/human-turn1.grammar-tool-output.csv'